# Data Loading Module

The present notebook reviews the **dataloading** module from the *DL* package of the **QQuantLib** library (**QQuantLib/DL/data_loading.py**). This module focuses on loading data into quantum states or circuits, enabling the preparation of quantum states that correspond to specific probability distributions.

The content of this notebook and the associated module are based on the following references:

- **Grover, Lov, and Rudolph, Terry**. *Creating superpositions that correspond to efficiently integrable probability distributions*. arXiv (2002).  
  [https://arxiv.org/abs/quant-ph/0208112](https://arxiv.org/abs/quant-ph/0208112)

- **V.V. Shende, S.S. Bullock, and I.L. Markov**. *Synthesis of quantum-logic circuits*. IEEE Transactions on Computer-Aided Design of Integrated Circuits and Systems, 25(6):1000–1010, Jun 2006.  
  [https://arxiv.org/abs/quant-ph/0406176v5](https://arxiv.org/abs/quant-ph/0406176v5)

- **NEASQC Deliverable**: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR*.  
  [https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf](https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf)

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
#This cell loads the QLM solver. QPU = [qlmass, python, c]
from QQuantLib.qpu.get_qpu import get_qpu
# myqlm qpus: python, c
# QLM qpus accessed using Qaptiva Access library: qlmass_linalg, qlmass_mps
# QLM qpus: Only in local Quantum Learning Machine: linalg, mps
my_qpus = ["python", "c", "qlmass_linalg", "qlmass_mps", "linalg", "mps"]

linalg_qpu = get_qpu(my_qpus[1])

The following cell loads the `get_results` function from **QQuantLib/utils/data_extracting**. This function is frequently used in the current library, so we will explain its purpose, inputs, and outputs.

The `get_results` function accepts a QLM object (a QLM Gate, QRoutine, or Program) and creates the associated QLM program, circuit, and job. It then simulates the quantum program, retrieves the results, and post-processes them.

---

### Inputs for `get_results`:
- `quantum_object`: A QLM Gate, Routine, or Program.
- `linalg_qpu`: A QLM solver (e.g., a Quantum Processing Unit).
- `shots`: An integer specifying the number of shots for the generated job.
- `qubits`: A list of qubits to measure during simulation.
- `complete`: A boolean flag. If `True`, returns the complete basis states, which is useful when `shots` is not zero and all possible basis states are required.

---

### Outputs of `get_results`:
- `pdf`: A Pandas DataFrame containing the results of the simulation.
- `circuit`: The QLM circuit corresponding to the input QLM object.
- `q_prog`: The QLM Program corresponding to the input QLM object.
- `job`: The QLM Job corresponding to the input QLM object.

The primary output of this function is the Pandas DataFrame (`pdf`). The columns provided in the DataFrame are as follows:

- `States`: Possible quantum states resulting from measurements on the circuit.
- `Int_lsb`: Conversion of the quantum state to an integer using **lsb** encoding (the bit farthest to the right is the least significant).
- `Probability`: Computed frequency of the quantum state when `shots` is not zero. When `shots` is zero, theoretical probabilities are provided.
- `Amplitude`: Amplitude of the quantum states (only provided if `shots` is zero).
- `Int`: Conversion of the quantum state to an integer using standard encoding (the bit farthest to the right is the most significant).

This function simplifies the process of simulating quantum circuits and extracting meaningful results, making it a valuable tool for analyzing quantum computations.

In [ ]:
from QQuantLib.utils.data_extracting import get_results

## 1. Loading Data

Typically, when loading data into a quantum circuit, the goal is to encode a discrete probability distribution $ p_d $ and an array $ f $. The first step is to define the dimension of the circuit and specify the data to be loaded. Here, $ n $ represents the number of qubits, and $ N = 2^n $ denotes the size of the discretized probability distribution as well as the size of the array.

In this specific example, we set $ n = 3 $, which results in $ N = 8 $. This means the circuit will consist of 3 qubits, and both the probability distribution and the array will have 8 elements.

In [ ]:
n = 3
N = 2**n
x = np.arange(N)

Next, we define a discrete probability distribution:
$$
p_d = \left(p_0, p_1, p_2, p_3, p_4, p_5, p_6, p_7\right).
$$

In this specific example, we generate the following probability distribution:
$$
p_d = \frac{1}{0 + 1 + 2 + 3 + 4 + 5 + 6 + 7} \left(0, 1, 2, 3, 4, 5, 6, 7\right),
$$
which is stored in the variable `probability`.

In [ ]:
probability = x/np.sum(x)

Finally, we define an array:
$$
f = \left(f_0, f_1, f_2, f_3, f_4, f_5, f_6, f_7\right).
$$

Later, it will be useful to work with a normalized version of this function, denoted as $\hat{f}$. The key characteristic of $\hat{f}$ is that its maximum absolute value is scaled to 1, i.e., $||\hat{f}||_{\infty} = 1$:
$$
\hat{f} = \frac{f}{||f||_{\infty}} = \left(\hat{f}_0, \hat{f}_1, \hat{f}_2, \hat{f}_3, \hat{f}_4, \hat{f}_5, \hat{f}_6, \hat{f}_7\right).
$$

In the code, this normalization is implemented using the variable `normalization_constant`, which corresponds to $||f||_{\infty}$.

In this specific example, we choose $f$ to be:
$$
f = \left(0, 1, 2, 3, 4, 5, 6, 7\right).
$$

Thus, the normalized function $\hat{f}$ becomes:
$$
\hat{f} = \frac{f}{||f||_{\infty}} = \frac{1}{7} \left(0, 1, 2, 3, 4, 5, 6, 7\right).
$$

In [ ]:
normalization_constant = np.max(x)
f = x
f_normalised = x/normalization_constant

In [ ]:
%matplotlib inline
plt.plot(x, probability, 'o')
plt.plot(x, f_normalised, 'o')
plt.grid()
plt.legend(['Probability', 'Array to load'])

### 1.1 Loading Probability

In [ ]:
from QQuantLib.DL.data_loading import load_probability

To load a discrete probability distribution into a quantum state, the function `load_probability` from the **DL/data_loading** module can be used. The inputs to this function are as follows:

- **Probability Distribution (mandatory)**: A NumPy array containing the probability distribution to be loaded into the quantum state. In this case, the variable `probability` holds the desired probability distribution.
- `id_name`: A string that provides a name for the Abstract Gate created by the function. If no name is specified, the current CPU time will be appended to the gate's name.
- `method`: A string that specifies the internal implementation for the controlled rotation by state, which forms the basis of the data loading method. Two options are available:
  - `multiplexor`: Uses quantum multiplexors for data loading. This is the default option.
  - `brute_force`: Implements the controlled rotation by state directly, resulting in longer circuits.

The output of the function is a **qlm** `AbstractGate` with arity *n*, where *n* is the number of qubits in the circuit.

In [ ]:
#No id_name provided
routine = load_probability(probability)
#Display circuit representation
%qatdisplay routine --depth 0 --svg

Now, the quantum state has the following form:
$$
\sqrt{p_0}|0\rangle + \sqrt{p_1}|1\rangle + \sqrt{p_2}|2\rangle + \sqrt{p_3}|3\rangle + \sqrt{p_4}|4\rangle + \sqrt{p_5}|5\rangle + \sqrt{p_6}|6\rangle + \sqrt{p_7}|7\rangle
$$

Finally, we use the function `get_results` from the **data_extracting** module to retrieve the probabilities loaded into the quantum circuit. Due to the properties of quantum mechanics, the measurement outcomes correspond to the squared amplitudes of the quantum state:

$$
\left(\sqrt{p_0}^2, \sqrt{p_1}^2, \sqrt{p_2}^2, \sqrt{p_3}^2, \sqrt{p_4}^2, \sqrt{p_5}^2, \sqrt{p_6}^2, \sqrt{p_7}^2\right) = \left(p_0, p_1, p_2, p_3, p_4, p_5, p_6, p_7\right)
$$

This confirms that the original probability distribution $ \{p_0, p_1, \dots, p_7\} $ has been successfully encoded into the quantum state.

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values
print("Quantum probabilities: ",quantum_probabilities)
print("Classical probabilities: ",probability)
#Test
np.isclose(quantum_probabilities, probability).all()

### brute-force implementation

In [ ]:
#With id_name provided
routine_bf = load_probability(probability, id_name='brute_force', method = 'brute_force')
#Display circuit representation
%qatdisplay routine_bf --depth  --svg

In [ ]:
results,_,_,_ = get_results(routine_bf, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values
print("Quantum probabilities: ",quantum_probabilities)
print("Classical probabilities: ",probability)
#Test
np.isclose(quantum_probabilities, probability).all()

### quantum multiplexors implementation

In [ ]:
#With id_name provided
routine_mp = load_probability(probability, id_name='multiplexors', method = 'multiplexor')
#Display circuit representation
%qatdisplay routine_mp --depth  --svg

In [ ]:
results,_,_,_ = get_results(routine_mp, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values
print("Quantum probabilities: ",quantum_probabilities)
print("Classical probabilities: ",probability)
#Test
np.isclose(quantum_probabilities, probability).all()

As can be observed, the quantum multiplexors implementation results in a circuit with lower depth compared to alternative methods. This efficiency makes it a preferred choice for loading probability distributions into quantum states.

To ensure proper functionality, the `load_probabilities` function must be the first gate applied in the entire circuit. 

### 1.2 Loading Function

To load an array into a quantum circuit, additional steps are required compared to loading a probability distribution alone. First, a probability distribution must be loaded into the quantum state, and an extra qubit must be reserved for encoding the array values.

In this context, we will not delve into the details of loading a probability distribution, as it has already been covered in the previous subsection. Instead, we will use the `uniform_distribution` function from the **DL/data_loading** module to simplify this step. This function allows us to focus on the process of loading the array while assuming a uniform probability distribution for the quantum state.

In [ ]:
from QQuantLib.DL.data_loading import uniform_distribution

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
routine.apply(uniform_distribution(n),register[:n])

Our circuit has the form:

In [ ]:
%qatdisplay routine --depth 1 --svg

The state currently loaded into the circuit is:
$$
\frac{1}{\sqrt{N}} |0\rangle \left[ |0\rangle + |1\rangle + |2\rangle + |3\rangle + |4\rangle + |5\rangle + |6\rangle + |7\rangle \right]
$$

The next step is to load the array into the quantum state. For this purpose, the function `load_array` from the **DL/data_loading** module can be used. This function accepts the following arguments:

1. **Normalized Array**: An array with an infinity norm ($||f||_{\infty}$) equal to or less than $1$. This ensures that the array values are scaled appropriately for encoding into the quantum state.
   
2. **`method`**: Specifies the implementation approach for loading the array. By default, this argument is set to `multiplexors`, which is more efficient in terms of quantum gates. Alternatively, the value `brute_force` can be used, though it is significantly less efficient.

3. **`id_name`**: A string used to assign a name to the Abstract Gate created by the function. If no name is provided, the current CPU time will be appended to the gate's name.

This function enables the efficient encoding of the array into the quantum state, building upon the uniform probability distribution already loaded into the circuit.

In [ ]:
from QQuantLib.DL.data_loading import load_array

In [ ]:
#Not Providing id_name
load_gate = load_array(f_normalised)
%qatdisplay load_gate --depth 0 --svg

In [ ]:
#Providing id_name
load_gate = load_array(f_normalised, id_name='f')
%qatdisplay load_gate --depth 0 --svg

In [ ]:
#Now apply the gate to the routine
routine.apply(load_gate, register)


$$\dfrac{1}{\sqrt{N}}|0\rangle\left[\hat{f}_0|0\rangle+\hat{f}_1|1\rangle+\hat{f}_2|2\rangle+\hat{f}_3|3\rangle+\hat{f}_4|4\rangle+\hat{f}_5|5\rangle+\hat{f}_6|6\rangle+\hat{f}_7|7\rangle\right]+...$$
The associated circuit is:

In [ ]:
%qatdisplay routine --depth 0 --svg

Finally, we measure the probabilities that we have loaded using `get_results`function.

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

By the properties of quantum mechanics, the result stored in the variable `quantum_probabilities` corresponds to the absolute square of the normalized array $\hat{f}$, divided by $N$. Specifically, it is given by:
$$
p = \frac{1}{N} \left( \hat{f}_0^2, \hat{f}_1^2, \hat{f}_2^2, \hat{f}_3^2, \hat{f}_4^2, \hat{f}_5^2, \hat{f}_6^2, \hat{f}_7^2, \dots \right)
$$

While additional information may be stored in the variable, it is not required for the current analysis.

To recover the original function $ f $ from the probabilities stored in the variable `quantum_probabilities`, we need to compute the square root of each probability $ p_i $, multiply it by $ N $ (the size of the array), and scale it by the normalization constant $ ||f||_{\infty} $. The formula is as follows:

$$
f_i = N \cdot ||f||_{\infty} \cdot \sqrt{p_i}
$$

This process reverses the normalization applied during the encoding of $ f $ into the quantum state, allowing us to reconstruct the original array.

In [ ]:
quantum_f = np.sqrt(quantum_probabilities)*np.sqrt(N)*normalization_constant
print("Array loaded in the quantum state: ",quantum_f[:N])
print("Original array: ",f)
#Test
np.isclose(quantum_f[:N], f).all()

### 1.3 Loading a function upon a non trivial probability distribution

Now, we will load both a discrete probability distribution $ p_d $ and an array $ f $ simultaneously. This case combines the approaches from the two previous examples. 

First, we begin by loading the discrete probability distribution $ p_d $. Note that when loading the normalized array $ \hat{f} $, an additional qubit is required. The probability distribution $ p_d $ is encoded only in the first three registers of the quantum state.

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
routine.apply(load_probability(probability, id_name='p(x)'),register[:n])
%qatdisplay routine --depth 0 --svg

At this stage, our quantum state is represented as:
$$
|0\rangle \left[ \sqrt{p_0}|0\rangle + \sqrt{p_1}|1\rangle + \sqrt{p_2}|2\rangle + \sqrt{p_3}|3\rangle + \sqrt{p_4}|4\rangle + \sqrt{p_5}|5\rangle + \sqrt{p_6}|6\rangle + \sqrt{p_7}|7\rangle \right]
$$

Next, we compute the necessary angles and proceed to load the function into the quantum state. Instead of loading the normalized array $ \hat{f} $, we will load $ \sqrt{\hat{f}} $. This adjustment ensures that all components are consistently encoded within the same framework.

In [ ]:
f_root = np.sqrt(f_normalised)
routine.apply(load_array(f_root, id_name='f(x)'),register)
%qatdisplay routine --depth 0 --svg

At this point, our quantum state is represented as:
$$
|0\rangle \left[ \sqrt{p_0 \hat{f}_0}|0\rangle + \sqrt{p_1 \hat{f}_1}|1\rangle + \sqrt{p_2 \hat{f}_2}|2\rangle + \sqrt{p_3 \hat{f}_3}|3\rangle + \sqrt{p_4 \hat{f}_4}|4\rangle + \sqrt{p_5 \hat{f}_5}|5\rangle + \sqrt{p_6 \hat{f}_6}|6\rangle + \sqrt{p_7 \hat{f}_7}|7\rangle \right] + \dots
$$

If we perform a measurement on this state, the resulting probabilities can be compared to the element-wise product of the probability distribution $ p_d $ and the normalized array $ \hat{f} $. 

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

In [ ]:
quantum_result = quantum_probabilities*normalization_constant
print("Quantum result: ",quantum_result[0:N])
print("Classical result: ",probability*f)
np.isclose(quantum_result[0:N], probability*f).all()

If we wish to compute the scalar product using the previous technique, there is an elegant approach we can employ. By measuring only the last qubit (the one furthest to the left in the quantum state representation or at the bottom of the circuit), we effectively calculate the following quantity:

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu,qubits = [n])
quantum_probabilities = results["Probability"].values

In [ ]:
quantum_result = quantum_probabilities*normalization_constant
print("Quantum result: ",quantum_result[0])
print("Classical result: ",np.dot(probability,f))
#Test
np.isclose(quantum_result[0], np.dot(probability,f))

### 1.4 Loading two arrays

In our final example, we will load two arrays, $ f $ and $ g = p_d $. To encode two arrays into the quantum state, we require two additional qubits: one for the first array ($ f $) and another for the second array ($ g $). 

We begin by defining a base routine with a size of $ n + 2 $, where $ n $ is the number of qubits required to represent the original state. Since a base probability distribution must always be loaded, we initialize the state with a uniform probability distribution.

In [ ]:
g = probability
routine = qlm.QRoutine()
register = routine.new_wires(n+2)
routine.apply(uniform_distribution(n),register[:n])
%qatdisplay routine --depth 0 --svg

We have already defined the normalized version of $ f $, denoted as $ \hat{f} $, which is stored in the variable `f_normalised`. However, the discrete probability distribution $ p_d $ (denoted as $ g $) is not yet normalized. To address this, we define its normalized version as follows:

$$
\hat{g} = \frac{g}{||g||_{\infty}},
$$

where $ ||g||_{\infty} $ represents the infinity norm of $ g $, ensuring that the maximum absolute value of $ \hat{g} $ is equal to 1.

In [ ]:
g_normalised = g/np.max(g)

Now we have to load the normalised arrays in the circuit. Because we have two arrays instead of one, we are going to load the first array in the first reserved register and the second one in the second reserved register.

In [ ]:
routine.apply(load_array(f_normalised,id_name= '1'),register[0:n],register[n])
routine.apply(load_array(g_normalised,id_name= '2'),register[0:n],register[n+1])

In [ ]:
%qatdisplay routine --depth 0 --svg

Now our quantum state is:
$$\dfrac{1}{\sqrt{N}}|0\rangle|0\rangle\left[f_0 g_0|0\rangle+f_1 g_1|1\rangle+f_2 g_2|2\rangle+f_3 g_3|3\rangle+f_4 g_4|4\rangle+f_5 g_5|5\rangle+f_6 g_6|6\rangle+f_7 g_7|7\rangle\right]+...$$
If we measure again, we can compare our result with the element-wise product of $g$ and $f$

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

In [ ]:
quantum_result = np.sqrt(quantum_probabilities*N)*np.max(f)*np.max(g)
classical_result = f*g
print("Quantum result: ",quantum_result[:N])
print("Classical result: ",classical_result)
np.isclose(quantum_result[:N], classical_result).all()

To compute the scalar product from our quantum state, we can apply a Hadamard transform. The first coefficient of the Hadamard transform corresponds to the sum of the input vector, which represents our previous state:

$$
\frac{1}{\sqrt{N}} |0\rangle |0\rangle \left[ f_0 g_0 |0\rangle + f_1 g_1 |1\rangle + f_2 g_2 |2\rangle + f_3 g_3 |3\rangle + f_4 g_4 |4\rangle + f_5 g_5 |5\rangle + f_6 g_6 |6\rangle + f_7 g_7 |7\rangle \right] + \dots
$$

After applying the Hadamard transform, this state transforms into:

$$
\frac{1}{N} |0\rangle |0\rangle \left[ f_0 g_0 + f_1 g_1 + f_2 g_2 + f_3 g_3 + f_4 g_4 + f_5 g_5 + f_6 g_6 + f_7 g_7 \right] |0\rangle + \dots
$$

The remaining coefficients depend on the specific transformation being applied, as there are three distinct versions of this transformation. Note that, instead of the factor $\frac{1}{\sqrt{N}}$ present in the original state, the resulting state now includes a factor of $\frac{1}{N}$.

In [ ]:
routine.apply(uniform_distribution(n),register[:n])
%qatdisplay routine --depth 0 --svg

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

In [ ]:
quantum_result = np.sqrt(quantum_probabilities)*N*np.max(f)*np.max(g)
classical_result = np.dot(f,g)
print("Quantum result: ",quantum_result[0])
print("Classical result: ",classical_result)
#Test
np.isclose(quantum_result[0], classical_result)

### 1.5 Ordering conventions

In this context, we are following the qubit ordering convention $ |q_n \dots q_1 q_0\rangle $, where $ q_0 $ represents the least significant qubit. However, in the **QLM** framework, the opposite convention is used, where the state is represented as $ |q_0 q_1 \dots q_n\rangle $. Below, we demonstrate how to adapt our functions to align with the **QLM** convention, which is relatively straightforward.

To load a distribution while adhering to the **QLM** qubit ordering:

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n)
routine.apply(load_probability(probability),register[::-1])
%qatdisplay routine --depth 1 --svg

To get the results we do the same as always.

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)

Now we use the QLM ordering to get the probabilities: for this we can use the *Int* column from the *results* DataFrame

In [ ]:
quantum_probabilities = results.sort_values("Int")["Probability"].values
print("Quantum probabilities: ",quantum_probabilities)
print("Classical probabilities: ",probability)
np.isclose(quantum_probabilities, probability).all()

If we have to deal with the loading of functions it can be a little bit more tricky:

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
control = register[:n]
target = register[n]
routine.apply(uniform_distribution(n),register[:n])
routine.apply(load_array(f_normalised),[control[::-1],target])
%qatdisplay routine --depth 0 --svg

To retrieve the results it is less convenient as now the result is not in order 

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
results.sort_values("Int",inplace = True)
results["Function"] = np.sqrt(results["Probability"]*N)*normalization_constant
results

See that now the information is stored in all qubits marked with a zero in the leftmost qubit. That provoques that the information is stored only in the odd positions (with respect to the int convention).
Another possibility is applying everything in reverse:

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
control = register[1:n+1]
target = register[0]
routine.apply(uniform_distribution(n),register[1:n+1])
routine.apply(load_array(f_normalised),[control[::-1],target])
%qatdisplay routine --depth 0 --svg

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
results.sort_values("Int",inplace = True)
results["Function"] = np.sqrt(results["Probability"]*N)*normalization_constant
results

In this case the results are naturally stored in the order of QLM (the int order). 

## Appendix 1: Loading array with angles

In this appendix, we explain how to use the angles to load a properly normalized array $\hat{f}$ into a quantum circuit. Under the hood, loading an array involves encoding specific angles into the quantum state. As with previous examples, we begin by loading a base discrete probability distribution.

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(n+1)
routine.apply(uniform_distribution(n),register[:n])

To load the normalized array $\hat{f}$ into the quantum circuit using angles, we first need to compute the associated angles. These angles are calculated as follows:

$$
\theta = \left(\theta_0, \theta_1, \theta_2, \theta_3, \theta_4, \theta_5, \theta_6, \theta_7\right) = 2 \arccos\left(\hat{f}\right)
$$

Explicitly, this corresponds to:
$$
\theta = \left(2 \arccos\left(\hat{f}_0\right), 2 \arccos\left(\hat{f}_1\right), 2 \arccos\left(\hat{f}_2\right), 2 \arccos\left(\hat{f}_3\right), 2 \arccos\left(\hat{f}_4\right), 2 \arccos\left(\hat{f}_5\right), 2 \arccos\left(\hat{f}_6\right), 2 \arccos\left(\hat{f}_7\right)\right)
$$

This step highlights the importance of working with the normalized version of the function $f$, as it ensures that the computed angles are well-defined and suitable for encoding into the quantum state.

In [ ]:
angles = 2*np.arccos(f_normalised)

To load the angles into the quantum circuit, we use the function `load_angles` from the **data_loading** module. This function requires a NumPy array containing the angles associated with the normalized function $\hat{f}$ as input. In this case, the probability distribution is stored in the variable `probability`. The output of the function is a **qlm** `AbstractGate` with arity *n*, where *n* is the number of qubits in the circuit.

The `load_angles` function also accepts a second argument called `method`, which determines the implementation approach:
- By default, the `method` is set to `multiplexors`, which is more efficient in terms of quantum gates.
- Alternatively, the `method` can be set to `brute_force`, though this option is significantly less efficient and results in longer circuits.

In [ ]:
from QQuantLib.DL.data_loading import load_angles

In [ ]:
routine.apply(load_angles(angles),register)

Now, the quantum state has the following form:

$$
\frac{1}{\sqrt{N}} |0\rangle \left[ \cos\left(\frac{\theta_0}{2}\right)|0\rangle + \cos\left(\frac{\theta_1}{2}\right)|1\rangle + \cos\left(\frac{\theta_2}{2}\right)|2\rangle + \cos\left(\frac{\theta_3}{2}\right)|3\rangle + \cos\left(\frac{\theta_4}{2}\right)|4\rangle + \cos\left(\frac{\theta_5}{2}\right)|5\rangle + \cos\left(\frac{\theta_6}{2}\right)|6\rangle + \cos\left(\frac{\theta_7}{2}\right)|7\rangle \right] + \dots
$$

By substituting the computed values of the angles $\theta_i = 2 \arccos(\hat{f}_i)$, the state simplifies to:

$$
\frac{1}{\sqrt{N}} |0\rangle \left[ \hat{f}_0|0\rangle + \hat{f}_1|1\rangle + \hat{f}_2|2\rangle + \hat{f}_3|3\rangle + \hat{f}_4|4\rangle + \hat{f}_5|5\rangle + \hat{f}_6|6\rangle + \hat{f}_7|7\rangle \right] + \dots
$$

The corresponding quantum circuit for this state preparation is as follows:

In [ ]:
%qatdisplay routine --depth 0 --svg

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

Finally, we measure the probabilities that we have loaded and see that is the same as using the function **load_array**

In [ ]:
quantum_f = np.sqrt(quantum_probabilities)*np.sqrt(N)*normalization_constant
print("Array loaded in the quantum state: ",quantum_f[:N])
print("Original array: ",f)
np.isclose(quantum_f[:N], f).all()

## Appendix 2: Quantum Multiplexors

Implementation of data loading routines using the *Lov Grover and Terry Rudolph* routines directly, using controlled rotations by state, is highly inefficient. In general, the use of controlled rotations generates highly deep quantum circuits prone to errors. A more efficient approach is the use of Quantum Multiplexors.

A Quantum Multiplexor in our case is a routine that applies a quantum gate depending on a control register. To simplify things, we are interested in Quantum Multiplexed Ry gates. Specifically, a Quantum Multiplexed Ry gate applies a $Ry$ rotation to a specific qubit depending on a control register. 
To give an example we are going to focus on a register with four qubits, the first three ones are the controls and the fourth one is the target qubit:
$$
\begin{array}{l}
&|0000\rangle\longrightarrow |0\rangle|0\rangle\\
&|0001\rangle\longrightarrow |0\rangle|1\rangle\\
&|0010\rangle\longrightarrow |0\rangle|2\rangle\\
&|0011\rangle\longrightarrow |0\rangle|3\rangle\\
&|0100\rangle\longrightarrow |0\rangle|4\rangle\\
&|0101\rangle\longrightarrow |0\rangle|5\rangle\\
&|0110\rangle\longrightarrow |0\rangle|6\rangle\\
&|0111\rangle\longrightarrow |0\rangle|7\rangle\\
\end{array}
$$
A quantum Quantum Multiplexed Ry gate applies a $Ry$ gate to the target register with angles $(\theta_0,\theta_1,\theta_2,\theta_3,\theta_4,\theta_5,\theta_6,\theta_7)$. It is easier to visualize it:
$$
\begin{array}{l}
&|0\rangle|0\rangle\longrightarrow \cos(\theta_0)|0\rangle|0\rangle+\sin(\theta_0)|1\rangle|0\rangle\\
&|0\rangle|1\rangle\longrightarrow \cos(\theta_1)|0\rangle|1\rangle+\sin(\theta_1)|1\rangle|1\rangle\\
&|0\rangle|2\rangle\longrightarrow \cos(\theta_2)|0\rangle|2\rangle+\sin(\theta_2)|1\rangle|2\rangle\\
&|0\rangle|3\rangle\longrightarrow \cos(\theta_3)|0\rangle|3\rangle+\sin(\theta_3)|1\rangle|3\rangle\\
&|0\rangle|4\rangle\longrightarrow \cos(\theta_4)|0\rangle|4\rangle+\sin(\theta_4)|1\rangle|4\rangle\\
&|0\rangle|5\rangle\longrightarrow \cos(\theta_5)|0\rangle|5\rangle+\sin(\theta_5)|1\rangle|5\rangle\\
&|0\rangle|6\rangle\longrightarrow \cos(\theta_6)|0\rangle|6\rangle+\sin(\theta_6)|1\rangle|6\rangle\\
&|0\rangle|7\rangle\longrightarrow \cos(\theta_7)|0\rangle|7\rangle+\sin(\theta_7)|1\rangle|7\rangle\\
\end{array}
$$
This is the idea. Now, to implement this operator in the original paper they propose a recursive way. We prefer to propose a new method to do it without the need of recursion. In this way, it is easier to analyse the computational cost.

To do the implementation we just need to ingredients, CNOT gates and single rotations. The CNOT gates and the RY rotations have to be applied in an alternating order. See the image below.

In [ ]:
m = 3
angles = np.arange(2**m)
angles = angles/np.max(angles)
routine = qlm.QRoutine()
register = routine.new_wires(m+1)
routine.apply(load_angles(angles),register)
%qatdisplay routine --depth 0 --svg

See that the CNOT gates have always as a target the target register. The controls simply have $2^i$ periodicity with a certain $i$ depending on the position. In simpler words, the first CNOT (in the image above the one controlling in $q_2$, in the image below it controls) is in the positions $(0,2,4,6...)$. The second one is going to be in positions $(1+0,1+4,1+8,...)$. In the image below we can see a more extensive example:

In [ ]:
m = 4
angles = np.arange(2**m)
angles = angles/np.max(angles)
routine = qlm.QRoutine()
register = routine.new_wires(m+1)
routine.apply(load_angles(angles),register)
%qatdisplay routine --depth 0 --svg

Finally, the angles for the rotations are the Hadamard transform in sequence order of the angles that we want to load. For the circuit above it is possible to check it using the fast Walsh Hadamard transform:

In [ ]:
from QQuantLib.utils.utils import fwht

In [ ]:
fwht(angles)/2**m